In [1]:
import sys
sys.path.append("/home/roozbeh/Morphology")
import numpy as np
from numpy import genfromtxt
import pandas as pd
import McNeuron
import matplotlib.pyplot as plt
from copy import deepcopy
from numpy.linalg import inv
from collections import Counter
import pickle
from sympy.solvers import solve
from sympy import nsolve
from sympy import Symbol
from numpy import sqrt as sqrt
from McNeuron import asymmetric as asy
import scipy
import scipy.io as spio
%matplotlib inline
import Atree

# Making swc files from cynalder model of the trees

In [ ]:
# tree_data = Atree.data_util.pandas_non_missing()
# pickle.dump(tree_data, open('./trees4roozbeh/tree_data.csv', 'wb'))

In [2]:
tree_data = pickle.load(open('./trees4roozbeh/tree_data.csv', 'rb'))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xf0 in position 6: ordinal not in range(128)

In [ ]:
result = tree_data.sort_values(['location'])
result = result.reset_index(drop=True)

In [3]:
mat = spio.loadmat("./trees4roozbeh/1.mat", squeeze_me=True)

In [ ]:
mat['Rad'].shape, mat['Len'].shape

In [ ]:
index = 30
tree = McNeuron.Neuron(result['swc'][index])
McNeuron.visualize.plot_2D(tree,projection=np.array([[0,1,0], [0,0,1], [1,0,0]]))

In [ ]:
ten = np.load('/data/roozbeh_arch/All Data/ten_sample.npy')

meta_data_path = '/home/roozbeh/Neuron Morphology/All Data/meta.csv'
regular_swc_path = '/home/roozbeh/Neuron Morphology/All Data/regular_swc.npy'
meshed_5_swc_path = '/home/roozbeh/Neuron Morphology/All Data/meshed_5_swc.npy'
raw_swc_path = '/home/roozbeh/Neuron Morphology/All Data/raw_swc.npy'
morph_data  = McNeuron.data_util.read_data(meta_data_path=meta_data_path,
                                               swc_path=raw_swc_path)
morph_data_5  = McNeuron.data_util.read_data(meta_data_path=meta_data_path,
                                               swc_path=meshed_5_swc_path)
morph_data_r  = McNeuron.data_util.read_data(meta_data_path=meta_data_path,
                                               swc_path=regular_swc_path)

In [ ]:
def rall_power(self, power=1.5, branching_degree='all'):
    """
    Computing the Rall power. It looks at the branching and compute the ratio of
    Sum(diameter of children to the power) over the diameter of the mother. Notice
    that it if the number of children can be more than two.
    
    Parameters:
    -----------
    power: float
        the power to compute Rall power
        
    Returns:
    --------
    rall_power: numpy vector
        A vector with the size of the number of branching nodes
        
    """
    self.set_branch_order()
    if branching_degree is 'all':
        branch_index = np.where(self.features['branch order'][1:]>1)[0]
    else:
        branch_index = np.where(self.features['branch order'][1:]==branching_degree)[0]
    branch_index += 1
    n_branch = len(branch_index)
    rall_power = np.zeros(n_branch)
    for i in range(n_branch):
        index_branch = branch_index[i]
        children = np.where(self.parent_index==index_branch)[0]
        children_diam = self.diameter[children]
        mother_diam = self.diameter[index_branch]
        # print children_diam, mother_diam
        if power is 'max':
            rall_power[i] = mother_diam/np.max(children_diam)
        else:
            rall_power[i] = np.power(mother_diam, power)/np.power(children_diam, power).sum()
    rall_power[np.isnan(rall_power)] = -1
    rall_power[np.isinf(rall_power)] = -1
    return rall_power


In [ ]:
McNeuron.visualize.plot_2D(tree)
plt.plot(tree.diameter)
rall_power(tree, power=2, branching_degree='all')[:]

In [ ]:
index = 92
power = 1./3.
power = 2
hist = np.zeros(19)
for index in range(1000,2000):
    tree = McNeuron.Neuron(morph_data_r['swc'][index])
    #McNeuron.visualize.plot_2D(tree)
    h = np.histogram(rall_power(tree, power=power, branching_degree='all'), bins=np.arange(0,2,.1));
    hist = h[0] + hist
plt.plot(hist)

In [ ]:
index = 50
tree = McNeuron.Neuron(result['swc'][index])
tree = McNeuron.Neuron(ten[0])
tree = McNeuron.Neuron(morph_data_r['swc'][index])
plt.hist(rall_power(tree, power='max', branching_degree='all')[:], bins=np.arange(0,2,.1))
plt.show()
plt.hist(rall_power(tree, power=1, branching_degree='all')[:], bins=np.arange(0,2,.1))

In [ ]:
index = 2
tree = McNeuron.Neuron(result['swc'][index])
plt.hist(rall_power(tree,power=.5), bins=50)
plt.show()
plt.hist(rall_power(tree,power=1), bins=50)
plt.show()
plt.hist(rall_power(tree,power=1.5), bins=50)
plt.show()
plt.hist(rall_power(tree,power=2.5), bins=50)
plt.show()

In [ ]:
#McNeuron.visualize.plot_dendrogram(mat['CylData'][:,8].astype(int))
#plt.plot(mat['CylData'][:,8])

In [ ]:
for i in range(0,80,10):
    index = i
    tree = McNeuron.Neuron(result['swc'][index])
    print result['location'][index]
    McNeuron.visualize.plot_2D(tree,projection=np.array([[0,1,0], [0,0,1], [1,0,0]]))

In [ ]:
tree.basic_features()

In [ ]:
plt.plot(tree.discrepancy(np.arange(.1,4,.1)))

In [ ]:
L = np.array([])
for i in range(100):
    try:
        mat = spio.loadmat("./trees4roozbeh/"+str(i)+".mat", squeeze_me=True)
        parent = mat['CPar']
    except:
        L = np.append(L,i)

In [ ]:
reg = McNeuron.subsample.regular_subsample(A)
n2 = McNeuron.Neuron(reg)
n2.motif_features()

In [ ]:
n2.n_node, bb

In [ ]:
dd = n2.features['die die depth']
bd = n2.features['branch die depth']
bb = n2.features['branch branch depth']
plt.plot(bd**2/(4*dd*bb))

In [ ]:
n.features['branch die'], n.features['branch branch'], n.features['die die']

In [ ]:
n.features['branch order']>2

In [ ]:
McNeuron.visualize.plot_dendrogram(n2.parent_index[:400])

In [ ]:
plt.plot(n.branch_order(n.parent_index))

In [ ]:
n.features.keys()

In [ ]:
n.features['die die']

In [ ]:
branch_order = n.branch_order(n.parent_index)
branch_angle, side_angle = n.branch_angle(branch_order)
branch_angle

In [ ]:
plt.plot(n.features['branch depth'])

In [ ]:
bins = np.array([18, 18, 18, 28, 30, 100])
colors = ['red', 'blue', 'green', '#3CC2D9', '#FFA500', '#800080']
features = ['branch angle', 'local angle', 'global angle', 'distance from root',
          'segmental neural length', 'neuronal/euclidean for segments']
i = 5
a = np.histogram(n.features[features[i]],
                 bins =  np.arange(1,1.23,.01))[0].astype(float)
b = a/sum(a)
plt.bar(np.arange(b.shape[0]), 
        b, 
        width=.8, 
        color=colors[i])
#plt.axis('off')
plt.savefig('actual-tree-neuronal-euclidean for segments.eps')

In [ ]:
n.features['initial segments']

In [ ]:
plt.hist(n.features[features[i]], 30)

In [ ]:
a

In [ ]:
plt.hist(n.features['branch angle'])

In [ ]:
n.features['discrepancy space']

In [ ]:
plt.hist(n.features['branch order'])

In [ ]:
neuron = McNeuron.Neuron(data['X'][0])
vector = neuron.location - neuron.location[:, neuron.parent_index]

In [ ]:
BBdepth = n.features['branch branch depth']
BDdepth = n.features['branch die depth']
DDdepth = n.features['die die depth']
asy_index = ((BDdepth+1)**2).astype(float)/(4*(BBdepth+1)*(DDdepth+1)).astype(float)
plt.plot(asy_index[:])

In [ ]:
n.features['asymmetric ratio']

In [ ]:
n.features['branch die']

In [ ]:
n.features['die die']

In [ ]:
n.features['branch branch']

In [ ]:
data = pickle.load(open("Primary Cell Class.p", "rb" ))
data